<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/sir_dem_log_disc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display

# Método Contínuo com Termo Logístico
def sir_diff_logistic(t, y, beta1, beta2, gamma, nu, mu, alpha, K):
    S, I, R = y
    N = S + I + R
    # Evita N = 0 ou negativo com um limite inferior
    N = max(N, 1e-10)
    dS = N * (nu - N * (nu - mu) / K) - (beta1 / N + beta2) * S * I - mu * S
    dI = (beta1 / N + beta2) * S * I - gamma * I - (mu + alpha) * I
    dR = gamma * I - mu * R
    return [dS, dI, dR]

def solve_sir_logistic(beta1, beta2, gamma, nu, mu, alpha, N0, I0, R0, dt, T, K):
    S0 = max(0, N0 - I0 - R0)
    y0 = [S0, I0, R0]
    t_eval = np.arange(0, T + dt, dt)
    t_eval = t_eval[t_eval <= T]
    sol = solve_ivp(sir_diff_logistic, [0, T], y0, args=(beta1, beta2, gamma, nu, mu, alpha, K),
                    t_eval=t_eval, method='RK45', rtol=1e-6, atol=1e-8)
    N = sol.y[0] + sol.y[1] + sol.y[2]
    return sol.t, sol.y[0], sol.y[1], sol.y[2], N

# Função para atualizar os gráficos
def update_plot(beta1, beta2, gamma, nu, mu, alpha, N0, I0, R0, dt, T, K):
    t_cont, S_cont, I_cont, R_cont, N_cont = solve_sir_logistic(beta1, beta2, gamma, nu, mu, alpha, N0, I0, R0, dt, T, K)

    plt.figure(figsize=(8, 6), dpi=100)

    # t x N(t) Contínuo
    plt.subplot(2, 2, 1)
    plt.plot(t_cont, N_cont, linewidth=0.7)
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$N(t)$')
    plt.grid(True)
       # I(t) x N(t) Contínuo
    plt.subplot(2, 2, 2)
    plt.plot(I_cont, N_cont, linewidth=0.7, c='black')
    plt.xlabel('$I(t)$')
    plt.ylabel('$N(t)$')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Sliders com valores ajustados
beta1_slider = widgets.FloatSlider(value=0.32, min=0.0, max=1.0, step=0.000001, description='beta1:', readout_format='.6f')
beta2_slider = widgets.FloatSlider(value=0.002, min=0.0, max=1.0, step=0.000001, description='beta2:', readout_format='.6f')
gamma_slider = widgets.FloatSlider(value=0.129, min=0.0, max=1.0, step=0.000001, description='gamma:', readout_format='.6f')
nu_slider = widgets.FloatSlider(value=0.019, min=0.0, max=0.1, step=0.000001, description='nu:', readout_format='.6f')
mu_slider = widgets.FloatSlider(value=0.0074, min=0.0, max=0.1, step=0.000001, description='mu:', readout_format='.6f')
alpha_slider = widgets.FloatSlider(value=0.0320, min=0.0, max=0.1, step=0.000001, description='alpha:', readout_format='.6f')
N0_slider = widgets.IntSlider(value=10000, min=0, max=100000, step=1, description='N0:', readout_format='d')
I0_slider = widgets.IntSlider(value=10, min=0, max=1000, step=1, description='I0:', readout_format='d')
R0_slider = widgets.IntSlider(value=0, min=0, max=1000, step=1, description='R0:', readout_format='d')
dt_slider = widgets.FloatSlider(value=0.1, min=0.001, max=1.0, step=0.000001, description='dt:', readout_format='.6f')
T_slider = widgets.IntSlider(value=1*365, min=0, max=10*365, step=1, description='T:', readout_format='d')
K_slider = widgets.IntSlider(value=10300, min=1000, max=100000, step=100, description='K:', readout_format='d')

# Interface interativa com interactive_output e display
ui = widgets.VBox([
    beta1_slider, beta2_slider, gamma_slider, nu_slider, mu_slider, alpha_slider,
    N0_slider, I0_slider, R0_slider, dt_slider, T_slider, K_slider
])
out = widgets.interactive_output(update_plot, {
    'beta1': beta1_slider, 'beta2': beta2_slider, 'gamma': gamma_slider, 'nu': nu_slider,
    'mu': mu_slider, 'alpha': alpha_slider, 'N0': N0_slider, 'I0': I0_slider,
    'R0': R0_slider, 'dt': dt_slider, 'T': T_slider, 'K': K_slider
})
display(ui, out)

Output()